In [2]:
import numpy as np
import pandas as pd
import os
import math

In [3]:
results = pd.read_csv('Batch_4655371_batch_results.csv')
uids = pd.read_csv(
    '../final/youtubemix-unconditional-16s-exp-1/uids.txt', 
    sep=' ', 
    header=None, 
    names=['method', 'filename','uid'],
)

In [4]:
responses = []
for col in results.columns:
    if col.startswith('Input'):
        index = int(col.split("_")[-2])
        responses.append(
            results[
                [col, f'Answer.recording_{index}_musicality', f'Answer.recording_{index}_quality', 'WorkerId']
            ]
        )

In [5]:
responses = [e.rename(columns=dict(zip(e.columns, ['url', 'musicality', 'quality', 'worker']))) for e in responses]
responses = pd.concat(responses, axis=0)
responses['uid'] = responses['url'].apply(lambda x: x.split("/")[-1].replace(".wav", ""))
responses = responses[['uid', 'worker', 'quality', 'musicality']]

In [6]:
responses

,uid,worker,quality,musicality
0,21e150949efee464da90f534a23d4c9d,A175PJR0W3LO8I,2,3
1,21e150949efee464da90f534a23d4c9d,A18LFH7XW61JO9,4,2
2,21e150949efee464da90f534a23d4c9d,A1IHI23KH87K5W,3,2
3,21e150949efee464da90f534a23d4c9d,A1NF6PELRKACS9,3,2
4,21e150949efee464da90f534a23d4c9d,A1WY3YGT618GC0,3,2
...,...,...,...,...
595,d480865f9b38fe803042e325a28f5ab0,A3N7R7P9HP2YB6,3,2
596,d480865f9b38fe803042e325a28f5ab0,A3O2JUIF4DZQJU,2,2
597,d480865f9b38fe803042e325a28f5ab0,A3QZMGTVA4VO44,3,3
598,d480865f9b38fe803042e325a28f5ab0,AE861G0AY5RGT,4,3


In [7]:
data = pd.merge(responses, uids, on='uid', how='inner')
data = data.sort_values(['filename', 'method'])

In [8]:
data

,uid,worker,quality,musicality,method,filename
1140,ec3aa314da9bb01779c147c719a5711b,A18LFH7XW61JO9,4,2,samplernn-3,0.wav
1141,ec3aa314da9bb01779c147c719a5711b,A1IHI23KH87K5W,3,2,samplernn-3,0.wav
1142,ec3aa314da9bb01779c147c719a5711b,A1NF6PELRKACS9,3,1,samplernn-3,0.wav
1143,ec3aa314da9bb01779c147c719a5711b,A1S88VQY8G8CNC,3,1,samplernn-3,0.wav
1144,ec3aa314da9bb01779c147c719a5711b,A1WY3YGT618GC0,4,1,samplernn-3,0.wav
...,...,...,...,...,...,...
1415,12e0c8b2bad640fb19488dec4f65d4d9,A3N7R7P9HP2YB6,3,3,wavenet-1024,9.wav
1416,12e0c8b2bad640fb19488dec4f65d4d9,A3QZMGTVA4VO44,4,3,wavenet-1024,9.wav
1417,12e0c8b2bad640fb19488dec4f65d4d9,AA7BTDZQNHW5,4,4,wavenet-1024,9.wav
1418,12e0c8b2bad640fb19488dec4f65d4d9,AISFU4P55HYEH,2,3,wavenet-1024,9.wav


In [9]:
# Quality check: workers should not give ratings higher than those indicated
control_files = {
    '4.wav': (2, 2), 
    '10.wav': (2, 2), 
    '15.wav': (5, 2), 
    '17.wav': (2, 2), 
    '21.wav': (2, 2), 
}
blocklist = set()
for control, (expected_quality, expected_musicality) in control_files.items():
    df = data[(data['method'] == 'test') & (data['filename'] == control)][['worker', 'quality', 'musicality']]
    blocklist = blocklist.union(set(df[(df['quality'] > expected_quality) | (df['musicality'] > expected_musicality)]['worker']))
print("Discard Worker IDs:", blocklist)

Discard Worker IDs: {'A2OZ5SOGNN9QOA', 'A3M3HUU77NKTES', 'APSN3KV49VLKX', 'A2TG7EEQ2XG2IK', 'A38DC3BG1ZCVZ2', 'A1NOINYD1FZ55T'}


In [10]:
# Remove workers from data
clean_data = data[~data['worker'].isin(blocklist)]
# Remove test examples
data = data[~((data['method'] == 'test') & (data['filename'].isin(list(control_files.keys()))))]
clean_data = clean_data[~((clean_data['method'] == 'test') & (clean_data['filename'].isin(list(control_files.keys()))))]

In [11]:
clean_data.groupby(['method']).mean()

,quality,musicality
method,,
samplernn-3,2.976967,1.821497
sashimi-2,2.428023,2.420345
sashimi-8,2.854127,3.113244
test,3.755708,4.593607
wavenet-1024,2.909789,2.714012


In [12]:
def calc_stats(df, col):
    stats = df.groupby(['method'])[col].agg(['mean', 'count', 'std'])
    ci95_hi = []
    ci95_lo = []

    for i in stats.index:
        m, c, s = stats.loc[i]
        ci95_hi.append(m + 1.96*s/math.sqrt(c))
        ci95_lo.append(m - 1.96*s/math.sqrt(c))

    stats['ci95_hi'] = ci95_hi
    stats['ci95_lo'] = ci95_lo
    return stats

In [13]:
calc_stats(clean_data, 'quality').round(2)

,mean,count,std,ci95_hi,ci95_lo
method,,,,,
samplernn-3,2.98,521,0.98,3.06,2.89
sashimi-2,2.43,521,0.89,2.50,2.35
sashimi-8,2.85,521,0.96,2.94,2.77
test,3.76,438,0.95,3.84,3.67
wavenet-1024,2.91,521,0.92,2.99,2.83


In [14]:
calc_stats(clean_data, 'musicality').round(2)

,mean,count,std,ci95_hi,ci95_lo
method,,,,,
samplernn-3,1.82,521,0.88,1.90,1.75
sashimi-2,2.42,521,0.91,2.50,2.34
sashimi-8,3.11,521,0.95,3.20,3.03
test,4.59,438,0.70,4.66,4.53
wavenet-1024,2.71,521,0.85,2.79,2.64
